This notebook will show how to use the `gel` package to perform group elastic net regression. Suppose we have feature matrices $A_j$ ($j = 1 \dots p$), each of size $m \times n_j$ (i.e. $m$ $n_j$-dimensional samples), and targets $y$ (a $m$ dimensional vector). Group elastic net finds coefficients $\beta_j$ ($n_j$ dimensional vector), and $\beta_0$ (scalar) by solving the optimization problem

$$ \min_{\beta_0,\dots,\beta_p} \frac{1}{2}\left\|y - \beta_0 - \sum_{j=1}^p A_j\beta_j\right\|^2 + m\sum_{j=1}^p \sqrt{n_j}\left(\lambda_1\|\beta_j\| + \lambda_2\|\beta_j\|^2\right). $$

We will test this with some generated data.

In [1]:
m_tr = 100  # number of samples
ns = [10, 20, 5, 200, 1, 20]  # group sizes

# data distribution parameters
X_scale = 2.0
X_bias = 3.0

# coefficient distribution parameters
β_scale = 0.125
β_bias = 6.0

# true support
support = [0, 1, 0, 1, 1, 0]

In [2]:
# Generate data
import torch

A_trs = [torch.randn(m_tr, n_j)*X_scale + X_bias for n_j in ns]
βs = [s_j*(torch.randn(n_j, 1)*β_scale + β_bias) for s_j, n_j in zip(support, ns)]
y_tr = sum([A_j@β_j for A_j, β_j in zip(A_trs, βs)])[:, 0]

We will now use `gel` to solve group elastic net with a single pair of regularization values. There are two steps to this. First, `make_A` is used to convert the features as needed by the library. This, and other parameters are passed to gel_solve.

There are two implementations, `gelfista` and `gelcd`, which use proximal gradient descent and coordinate descent respectively. Refer to the respective source files for details on these methods. Here, we will use `gelfista`. `gelcd` is used in the same way.

In [9]:
from gel.gelcd import make_A, gel_solve, block_solve_newton

In [10]:
A_conv_tr = make_A(
    A_trs,
    torch.tensor(ns),  # note that this needs to be a LongTensor
    device=torch.device("cpu"),
    dtype=torch.float32,
)


In [11]:
# print(A_conv_tr.shape)

In [12]:
# print(y_tr.shape)  # note that y should be one dimensional

In [13]:
l_1 = 1.0  # λ_1
l_2 = 0.1  # λ_2

In [14]:
b_0, B = gel_solve(
    A_conv_tr,
    y_tr,
    l_1,
    l_2,
    ns,
    b_init=None,     # initialization value - useful if solving multiple instances
    block_solve_fun=block_solve_newton,
    # block_solve_kwargs=None,
    # max_cd_iters=None,
    # rel_tol=1e-6,
    # Cs=None,
    # Is=None,
    verbose=True,
)

TypeError: 'NoneType' object is not subscriptable

The returned `b_0` is a scalar, and `B` is a matrix. Each row of `B` is the coefficient vector for the corresponding group, padded with 0s. So we will first extract the vectors.

In [ ]:
βhats = [B[j, :ns[j]] for j in range(len(ns))]

In [ ]:
print(b_0)

In [ ]:
print([βhat_j.norm() for βhat_j in βhats])

This is the solution for a single (l_1, l_2) pair. But practically, we want to use a grid of values. So now, we will use the `gelpaths` module to sweep over multiple regularization values and pick the best based on validation results.

In [ ]:
# First, we generate validation data.
m_val = 30
A_vals = [torch.randn(m_val, n_j)*X_scale + X_bias for n_j in ns]
y_val = sum([A_j@β_j for A_j, β_j in zip(A_vals, βs)])[:, 0]

In [ ]:
# We need to standardize the data to prevent repeated computations.
A_tr_μs = [A_j.mean(dim=0, keepdim=True) for A_j in A_trs]
A_tr_σs = [A_j.std(dim=0, keepdim=True) for A_j in A_trs]
A_tr_stans = [(A_j - A_μ_j) / A_σ_j for A_j, A_μ_j, A_σ_j in zip(A_trs, A_tr_μs, A_tr_σs)]
A_val_stans = [(A_j - A_μ_j) / A_σ_j for A_j, A_μ_j, A_σ_j in zip(A_vals, A_tr_μs, A_tr_σs)]

y_tr_μ = y_tr.mean()
y_tr_σ = y_tr.std()
y_tr_stan = (y_tr - y_tr_μ) / y_tr_σ

`gelpaths` follows a 2-step procedure. For each (l_1, l_2) pair, the corresponding group elastic net problem is solved; this is then used to find the support by finding non-zero coefficient vectors. The support is then used to solve ridge regression over a range of regularization values.

For each ridge solution, a summary function is called. This is where we will compute the validation error, to find the optimal regularization values. Note that the summary function expects the features as a single matrix. 

In [ ]:
X_val = torch.cat(A_val_stans, dim=1).t()  # note the transpose

def summary(_support, _b):
    if _support is None:
        # empty support, so just use the computed training mean
        yhat_val = y_tr_μ
        _support_set = {}
    else:
        X_val_supp = X_val[_support]
        yhat_val = (X_val_supp.t() @ _b) * y_tr_σ + y_tr_μ  # we have to rescale
        _support_set = set([s.item() for s in _support])
        
    rmse = ((y_val - yhat_val) ** 2).mean().sqrt().item()

    # compute group support
    group_support = []
    start_idx = 0
    for n_j in ns:
        group_idxs = range(start_idx, start_idx + n_j)
        if all(i in _support_set for i in group_idxs):
            group_support.append(1)
        elif not any(i in _support_set for i in group_idxs):
            group_support.append(0)
        else:
            # This shouldn't happen! Groups are either all in or all out.
            raise AssertionError
        start_idx += n_j

    return group_support, rmse

There are two implementations in `gelpaths`. `gel_paths` requires manually specifying regularization values, and `gel_paths2` picks them automatically. We will use `gel_paths2` here.

In [ ]:
from gel.gelpaths import gel_paths2

l_rs = torch.logspace(-10, 10, 100).tolist()  # regularization strengths for ridge regression

summaries = gel_paths2(
    gel_solve,
    {
        # arguments to gel_solve
        "t_init": None,
        "ls_beta": 0.99,
        "max_iters": 3000,
        "rel_tol": 1e-5,
    },
    make_A,                                   # we don't explicitly call make_A here
    A_tr_stans,                               # we need to pass the standardized A values
    y_tr_stan,                                # note: standardized ouput vector
    ks=[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99], # this is a list of values to trade-off
                                              #   between l_1 and l_2 regularizations, with
                                              #   l_1 = k*l and l_2 = (1 - k)*l
    n_ls=50,                                  # number of l values for each k value
    l_eps=1e-5,                               # ratio of minimum to maximum l value;
                                              #    n_ls number of l values are generated
                                              #    according to this ratio, for each k
    l_rs=l_rs,                                # ridge regularization values
    summ_fun=summary,                         # the summary function
    supp_thresh=1e-4,                         # norm threshold for considering coefficient vectors
                                              #     to be 0 (default 1e-6)
    device=torch.device("cpu"),               # pytorch device (default cpu)
    verbose=True,                             # verbosity (default False)
    ls_grid=None,                             # override l value computation with a fixed grid
                                              #     (default is None)
    aux_rel_tol=1e-3,                         # tolerance for solving auxiliary problems
                                              #     to get better initializations (default 1e-3)
    dtype=torch.float32,                      # torch dtype (default float32)
)

Let us analyze the results.

In [ ]:
best_summ = min(summaries.items(), key=lambda z: z[1][1])
print(best_summ)

In [ ]:
best_support_hat = best_summ[1][0]
print(best_support_hat)

We were able to correctly identify the support! Now, we can perform a more thorough ridge regression on the identified support. For this we will use the `ridgepaths` module.

In [ ]:
X_tr_best_supp_hat = torch.cat(
    [A_j for A_j, s_j in zip(A_tr_stans, best_support_hat) if s_j == 1],
    dim=1,
).t()

X_val_best_supp_hat = torch.cat(
    [A_j for A_j, s_j in zip(A_val_stans, best_support_hat) if s_j == 1],
    dim=1,
).t()

def summary_best_supp_hat(support, b):
    # support is ignored
    yhat_val = (X_val_best_supp_hat.t() @ b) * y_tr_σ + y_tr_μ
    rmse = ((y_val - yhat_val) ** 2).mean().sqrt().item()
    return rmse

In [ ]:
from gel.ridgepaths import ridge_paths

l_rs_big = torch.logspace(-20, 20, 10000).tolist()

ridge_summaries = ridge_paths(
    X_tr_best_supp_hat,  # X needs to be pre-indexed using the support
    y_tr_stan,
    1,                   # this doesn't matter in this case since it is ignored; just shouldn't be None
    l_rs_big,
    summary_best_supp_hat,
    verbose=True,
)

In [ ]:
best_ridge_summ = min(ridge_summaries.items(), key=lambda z: z[1])
print(best_ridge_summ)

The validation RMSE doesn't change much, but the identified best lambda value _has_ changed. So now, using the identified support and ridge regularization value, we can train a final ridge regression model using the training and validation data combined to further increase performance. This can be tested using a separate test set if available.

In [ ]:
def dummy_summary(support, b):
    # Just return b
    return b

In [ ]:
# Combine training and validation data.
A_trvals = [torch.cat([A_j_tr, A_j_val], dim=0) for A_j_tr, A_j_val in zip(A_trs, A_vals)]
y_trval = torch.cat([y_tr, y_val])

# Compute combined statistics.
A_trval_μs = [A_j.mean(dim=0, keepdim=True) for A_j in A_trvals]
A_trval_σs = [A_j.std(dim=0, keepdim=True) for A_j in A_trvals]
y_trval_μ = y_trval.mean().item()
y_trval_σ = y_trval.std().item()

# Standardize data using combined statistics.
A_trval_stans = [(A_j - A_j_μ) / A_j_σ for A_j, A_j_μ, A_j_σ in zip(A_trvals, A_trval_μs, A_trval_σs)]
y_trval_stan = (y_trval - y_trval_μ) / y_trval_σ

# Project to support.
X_trval_best_supp_hat = torch.cat([
    A_j_trval
    for A_j_trval, s_j in zip(A_trval_stans, best_support_hat)
    if s_j == 1
], dim=1).t()
print(X_trval_best_supp_hat.shape)

In [ ]:
best_lambda = best_ridge_summ[0]
final_summary = ridge_paths(
    X_trval_best_supp_hat,
    y_trval_stan,
    1,
    [best_lambda],
    dummy_summary,
)
final_b = final_summary[best_lambda]

Here, since we know the true β, we can evaluate by comparing to it. We will assume knowledge of the fact that all features have the same normal distribution, and compute how well we estimate its parameters.

In [ ]:
from math import sqrt

X_trval = torch.cat(A_trvals, dim=1)
X_bias_hat = X_trval.mean().item()
X_scale_hat = X_trval.std().item()

supplen = sum(n_j for n_j, s_j in zip(ns, best_support_hat) if s_j == 1)
β_bias_hat = y_trval_μ / (X_bias_hat * supplen)
β_scale_hat = sqrt((((y_trval_σ ** 2) / supplen) - (β_bias_hat * X_scale_hat) ** 2) / ((X_bias_hat ** 2) + (X_scale_hat ** 2)))

In [ ]:
X_scale_hat, X_bias_hat  # true values (2, 3)

In [ ]:
β_scale_hat, β_bias_hat  # true values (0.125, 6)

We are able to estimate the parameters reasonably well, with most of the error in the scale parameter. And it should be noted again, that we did perfectly recover the support. To conclude, we compute the error on the combined training validation data.

In [ ]:
yhat_trval = y_trval_σ * (X_trval_best_supp_hat.t() @ final_b) + y_trval_μ
rmse = ((yhat_trval - y_trval) ** 2).mean().sqrt().item()
print(rmse)